In [1]:
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from torchtext.legacy.data import Iterator
import dill
import os
import time
import sys

sys.path.append('.')
from Data import MyTextDataset, get_field, get_pre_vector
from utils_dl import train_model, test_model
from language_model.FastText import Model
from utils import path

import warnings

warnings.filterwarnings("ignore")

path = path.rsplit('/', 1)[0]


def get_parameter_number(model):
    """获取模型藏书量

    Args:
        model (_type_): pytorch模型

    Returns:
        ditc: 总参数量和可训练参数量的字典
    """
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters()
                        if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}


In [2]:
#获取数据集路径
print('receiving data path...')
train_data = os.path.join(path, 'data', 'train_set')
validation_data = os.path.join(path, 'data', 'validation_set')
test_data = os.path.join(path, 'data', 'test_set')
#读取数据
print('reading data...')
start_time = time.time()
with open(train_data, 'rb') as f:
    train_example = dill.load(f)
with open(validation_data, 'rb') as f:
    validation_example = dill.load(f)
with open(test_data, 'rb') as f:
    test_example = dill.load(f)
end_time = time.time()
print('data reading time:', end_time - start_time)
#定义字段
print('defining fields...')
NEWS, LAST_NEWS_PERIOD, LABEL = get_field()
fields = [('news', NEWS), ('news_period', LAST_NEWS_PERIOD),
          ("label", LABEL)]
#构建数据,不单独区分测试集了
print('building dataset...')
train_set = MyTextDataset(examples=train_example + validation_example,
                          fields=fields)
test_set = MyTextDataset(examples=test_example, fields=fields)
#获取预训练词向量
print('receiving pretrained vectors...')
vectors = get_pre_vector() #注Vectors源文件做了修改，见https://blog.csdn.net/qq_23262411/article/details/100173224
#定义词表
print('define vocab...')
time3 = time.time()
NEWS.build_vocab(train_set, test_set, vectors=vectors,
                 min_freq=10)  #去除频率小于十次的词
#NEWS.vocab.vectors.unk_init = init.xavier_uniform_
weight_vector = NEWS.vocab.vectors
time4 = time.time()
print('building vocab time:', time4 - time3)
print('矩阵的维度为', weight_vector.shape)
print('矩阵的前十行为', weight_vector[:10])
#生成迭代器
print('define iterators...')
train_iter = Iterator(train_set,
                     batch_size=4096,
                     device=-1,
                     sort_key=lambda x: len(x.news),
                     sort_within_batch=False,
                     repeat=False)
test_iter = Iterator(test_set,
                    batch_size=4096,
                    device=-1,
                    sort=False,
                    sort_within_batch=False,
                    repeat=False)
#定义模型参数
print('define model parameter...')
n_vocab = len(NEWS.vocab)
embed_dims = 300
hidden_size = 256
num_classes = 2
weight_matrix = weight_vector
model_name = 'FastText'
n_epochs = 1 #先小点做测试
lr_ = 0.02
weight_decay_ = 0.0002
model_path = '/new_python_for_gnn/毕设code/model_cache'
#模型实例化
print('define model...')
fasttext = Model(n_vocab=n_vocab,
                 embed_dims=embed_dims,
                 hidden_size=hidden_size,
                 num_classes=num_classes,
                 weight_matrix=weight_matrix)
print('模型结构:')
print(fasttext)
parameters_num = get_parameter_number(fasttext)
print('模型总参数量为:')
print('total:{},size:{}M'.format(parameters_num['Total'],
                                 parameters_num['Total'] / 1e6))
print('模型可训练参数量为:')
print('train:{},size:{}M'.format(parameters_num['Trainable'],
                                 parameters_num['Trainable'] / 1e6))
#定义训练部件
print('define optimizer and criterion...')
optimizer = optim.Adam(fasttext.parameters(), lr=lr_, weight_decay=weight_decay_)
criterion = F.cross_entropy
#开始训练
print('begin training...')

receiving data path...
reading data...
data reading time: 1004.4600706100464
defining fields...
building dataset...
receiving pretrained vectors...
define vocab...


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


building vocab time: 769.8867354393005
矩阵的维度为 torch.Size([445247, 300])
矩阵的前十行为 tensor([[ 0.1696,  0.0177,  0.0050,  ...,  0.0498, -0.0477,  0.2002],
        [ 0.0625,  0.0278,  0.0178,  ..., -0.0122, -0.0442, -0.0020],
        [ 0.0786, -0.0027, -0.1833,  ...,  0.1627, -0.1661,  0.0388],
        ...,
        [ 0.0633, -0.1805, -0.0417,  ...,  0.0109, -0.1885,  0.2579],
        [-0.1469,  0.2031, -0.0347,  ...,  0.2687, -0.3711,  0.0732],
        [ 0.2368, -0.0158,  0.2339,  ..., -0.2937, -0.1052,  0.0433]])
define iterators...
define model parameter...
define model...
模型结构:
Model(
  (embedding): Embedding(445247, 300)
  (fc): Linear(in_features=300, out_features=300, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=300, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)
模型总参数量为:
total:133741970,size:133.74197M
模型可训练参数量为:
train:133741970,size:133.74197M
define optimizer 

In [3]:
import torch
model = fasttext

In [5]:
model.train()
device = torch.device('cuda:0')
model.to(device)
model = torch.nn.DataParallel(model)
total_step = len(train_iter)
print("            =======  Training  ======= \n")
train_loss_ = []  #一个epoch的训练误差
correct_ = []  #一个epoch的精度
accuracy_ = []

            =======  Training  ======= 



In [10]:
train_loss = 0
correct = 0
total = 0
train_loss += 12.5
train_loss

12.5

In [6]:
for epoch in range(n_epochs):
    train_loss = 0
    correct = 0
    total = 0
    for i, batch in enumerate(train_iter):
        data, labels = batch.news, batch.label
        period = batch.news_period
        #print(period)
        data = data.cuda()
        labels = labels.cuda().long()
        period = period.cuda()
        #print(data.shape) [1024,1000]
        #print(labels.shape) [1024]
        #print(period.shape) [1024]
        # Forward
        outputs = model(data, period)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #train_loss += loss.item()
        train_loss += float(loss)
        total += labels.size(0)
        correct += torch.eq(outputs.argmax(dim=1), labels).sum().item()
        accuracy = 100.0 * correct / total
        if (i + 1) % 25 == 0 or (i + 1) == total_step:
            print(
                'Epoch: [{:3}/{}], Step: [{:3}/{}], Ave_Loss: {:.3f}, acc: {:6.3f}'
                .format(
                    epoch + 1,
                    n_epochs,
                    i + 1,
                    total_step,
                    train_loss / (i + 1),
                    accuracy,
                ))
    train_loss_.append(train_loss)
    correct_.append(correct)
    accuracy_.append(accuracy)

Epoch: [  1/1], Step: [ 25/267], Ave_Loss: 10.178, acc: 50.236
Epoch: [  1/1], Step: [ 50/267], Ave_Loss: 7.716, acc: 50.882
Epoch: [  1/1], Step: [ 75/267], Ave_Loss: 5.411, acc: 51.458
Epoch: [  1/1], Step: [100/267], Ave_Loss: 4.234, acc: 51.755
Epoch: [  1/1], Step: [125/267], Ave_Loss: 3.553, acc: 51.911
Epoch: [  1/1], Step: [150/267], Ave_Loss: 3.094, acc: 52.066
Epoch: [  1/1], Step: [175/267], Ave_Loss: 2.752, acc: 52.148
Epoch: [  1/1], Step: [200/267], Ave_Loss: 2.494, acc: 52.208
Epoch: [  1/1], Step: [225/267], Ave_Loss: 2.294, acc: 52.260
Epoch: [  1/1], Step: [250/267], Ave_Loss: 2.134, acc: 52.269
Epoch: [  1/1], Step: [267/267], Ave_Loss: 2.042, acc: 52.294


In [9]:
train_loss

545.311806678772

In [11]:
print("\n            =======  Training Finished  ======= \n")


            =======  Training Finished  ======= 



In [13]:
PATH = os.path.join('/new_python_for_gnn/毕设code/model_cache', model_name + '.pt')
if not os.path.exists(PATH):
    torch.save(model.state_dict(), PATH)
print("\nModel saved... \n")


Model saved... 



In [15]:
from utils_dl import test_model
test_model(fasttext,'/new_python_for_gnn/毕设code/model_cache/FastText.pt',test_iter,'Fasttext')


            =======  Testing  ======= 

test accuracy 0.538, test macro f1-score 0.350

            =======  Testing Finished  ======= 


  calculating classification index...  

test_result的混淆矩阵为:

[[     0 125162]
 [     0 145636]]
准确率为0.5378030856948722
精确率为0.5378030856948722
召回率为1.0
特异度为0.0
假报警率为1.0
真正率为1.0
f1分数为0.6994433691773486
g_mean值为0.0
成功保存测试结果！

  finished saving index file...  



In [30]:
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from torchtext.legacy.data import Iterator
from torch.utils.data import DataLoader
import dill
import os
import time
import sys
sys.path.append('.')
from Data import MyTextDataset, get_field, get_pre_vector, PriceDataset
from utils_dl import train_model, test_model
from language_model.FastText import Model
from utils import path
import warnings
warnings.filterwarnings("ignore")
path = path.rsplit('/', 1)[0]
def get_parameter_number(model):
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters()
                        if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}


In [2]:
print('receiving data path...')
train_data = os.path.join(path, 'data', 'train_set')
validation_data = os.path.join(path, 'data', 'validation_set')
test_data = os.path.join(path, 'data', 'test_set')
#读取数据
print('reading data...')
start_time = time.time()
with open(train_data, 'rb') as f:
    train_example = dill.load(f)
with open(validation_data, 'rb') as f:
    validation_example = dill.load(f)
with open(test_data, 'rb') as f:
    test_example = dill.load(f)
end_time = time.time()
print('data reading time:', end_time - start_time)

receiving data path...
reading data...
data reading time: 1046.9419722557068


In [3]:
print('defining fields...')
NEWS, LAST_NEWS_PERIOD, LABEL = get_field()
fields = [('news', NEWS), ('news_period', LAST_NEWS_PERIOD),
          ("label", LABEL)]
#构建数据,不单独区分测试集了
print('building dataset...')
train_set = MyTextDataset(examples=train_example + validation_example,
                          fields=fields)
test_set = MyTextDataset(examples=test_example, fields=fields)
#获取预训练词向量
print('receiving pretrained vectors...')
vectors = get_pre_vector()  #注Vectors源文件做了修改，见https://blog.csdn.net/qq_23262411/article/details/100173224

defining fields...
building dataset...
receiving pretrained vectors...


In [4]:
#定义词表
print('define vocab...')
time3 = time.time()
NEWS.build_vocab(train_set, test_set, vectors=vectors,
                 min_freq=10)  #去除频率小于十次的词
#NEWS.vocab.vectors.unk_init = init.xavier_uniform_
weight_vector = NEWS.vocab.vectors
time4 = time.time()
print('building vocab time:', time4 - time3)
print('矩阵的维度为', weight_vector.shape)

define vocab...
building vocab time: 749.8627281188965
矩阵的维度为 torch.Size([445247, 300])


In [5]:
print('define iterators...')
train_iter = Iterator(train_set,
                      batch_size=4096,
                      device=-1,
                      sort_key=lambda x: len(x.news),
                      sort_within_batch=False,
                      shuffle=False,
                      repeat=False)
test_iter = Iterator(test_set,
                     batch_size=4096,
                     device=-1,
                     sort=False,
                     sort_within_batch=False,
                     shuffle=False,
                     repeat=False)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


define iterators...


In [6]:
print('price dataset...')
train_price = PriceDataset(train_or_test='train')
test_price = PriceDataset(train_or_test='test')
#构建迭代器
print('price iterators...')
train_price_iter = DataLoader(train_price,
                              batch_size=4096,
                              shuffle=False)
test_price_iter = DataLoader(test_price,
                             batch_size=4096,
                             shuffle=False)

price dataset...
price iterators...


In [19]:
print('define model parameter...')
n_vocab = len(NEWS.vocab)
embed_dims = 300
hidden_size = 256
num_classes = 2
weight_matrix = weight_vector
model_name = 'FastText'
n_epochs = 1  #先小点做测试
lr_ = 0.02
weight_decay_ = 0.0002
model_path = '/new_python_for_gnn/毕设code/model_cache'

define model parameter...


In [20]:
print('define model...')
fasttext = Model(n_vocab=n_vocab,
                 embed_dims=embed_dims,
                 hidden_size=hidden_size,
                 num_classes=num_classes,
                 weight_matrix=weight_matrix)
print('模型结构:')
print(fasttext)
parameters_num = get_parameter_number(fasttext)
print('模型总参数量为:')
print('total:{},size:{}M'.format(parameters_num['Total'],
                                 parameters_num['Total'] / 1e6))
print('模型可训练参数量为:')
print('train:{},size:{}M'.format(parameters_num['Trainable'],
                                 parameters_num['Trainable'] / 1e6))
#定义训练部件
print('define optimizer and criterion...')
optimizer = optim.Adam(fasttext.parameters(),
                       lr=lr_,
                       weight_decay=weight_decay_)
criterion = F.cross_entropy

define model...
模型结构:
Model(
  (embedding): Embedding(445247, 300)
  (fc): Linear(in_features=300, out_features=300, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=304, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)
模型总参数量为:
total:133742994,size:133.742994M
模型可训练参数量为:
train:133742994,size:133.742994M
define optimizer and criterion...


In [22]:
model =fasttext

In [24]:
import torch

In [25]:
model.train()
device = torch.device('cuda:0')
model.to(device)
model = torch.nn.DataParallel(model)
total_step = len(train_iter)
print("            =======  Training  ======= \n")
train_loss_ = []  #一个epoch的训练误差
correct_ = []  #一个epoch的精度
accuracy_ = []

            =======  Training  ======= 



In [26]:
for epoch in range(n_epochs):
    train_loss = 0
    correct = 0
    total = 0
    for i, (batch,price) in enumerate(zip(train_iter,train_price_iter)):
        data, labels = batch.news, batch.label
        period = batch.news_period
        #print(period)
        data = data.cuda()
        labels = labels.cuda().long()
        period = period.cuda()
        price = price.cuda().float()
        #print(data.shape) [1024,1000]
        #print(labels.shape) [1024]
        #print(period.shape) [1024]
        # Forward
        outputs = model(data, period,price)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #train_loss += loss.item()
        train_loss += float(loss)
        total += labels.size(0)
        correct += torch.eq(outputs.argmax(dim=1), labels).sum().item()
        accuracy = 100.0 * correct / total
        if (i + 1) % 25 == 0 or (i + 1) == total_step:
            print(
                'Epoch: [{:3}/{}], Step: [{:3}/{}], Ave_Loss: {:.3f}, acc: {:6.3f}'
                .format(
                    epoch + 1,
                    n_epochs,
                    i + 1,
                    total_step,
                    train_loss / (i + 1),
                    accuracy,
                ))
    train_loss_.append(train_loss)
    correct_.append(correct)
    accuracy_.append(accuracy)

Epoch: [  1/1], Step: [ 25/267], Ave_Loss: 0.806, acc: 50.895
Epoch: [  1/1], Step: [ 50/267], Ave_Loss: 0.811, acc: 51.593
Epoch: [  1/1], Step: [ 75/267], Ave_Loss: 0.772, acc: 52.173
Epoch: [  1/1], Step: [100/267], Ave_Loss: 0.753, acc: 52.275
Epoch: [  1/1], Step: [125/267], Ave_Loss: 0.744, acc: 52.159
Epoch: [  1/1], Step: [150/267], Ave_Loss: 0.736, acc: 51.316
Epoch: [  1/1], Step: [175/267], Ave_Loss: 0.743, acc: 51.691
Epoch: [  1/1], Step: [200/267], Ave_Loss: 0.737, acc: 51.369
Epoch: [  1/1], Step: [225/267], Ave_Loss: 0.732, acc: 51.217
Epoch: [  1/1], Step: [250/267], Ave_Loss: 0.728, acc: 51.720
Epoch: [  1/1], Step: [267/267], Ave_Loss: 0.726, acc: 51.823


In [28]:
import json
from collections import OrderedDict

print('\n loss saving... \n')
with open(
        os.path.join( '/new_python_for_gnn/毕设code/language_model_result/train',
                     model_name + '_loss.json'), 'wb') as f:
    loss_file = OrderedDict()
    loss_file['train_loss'] = train_loss_
    loss_file['correct'] = correct_
    loss_file['accuracy'] = accuracy_
    json_cont = json.dumps(loss_file, indent=4).encode(encoding='utf-8')
    f.write(json_cont)
print('\n success for loss saving...\n')
#保存模型方便之后进行模型测试等操作
print("\nModel saving... \n")
PATH = os.path.join( '/new_python_for_gnn/毕设code/model_cache', model_name + '.pt')
if not os.path.exists(PATH):
    torch.save(model.state_dict(), PATH)
print("\nModel saved... \n")


 loss saving... 


 success for loss saving...


Model saving... 


Model saved... 



In [21]:
print('begin training...')
time5 = time.time()
train_model(fasttext, train_iter, train_price_iter, optimizer, criterion,
            n_epochs, model_name)
time6 = time.time()
print('training time:', time6 - time5)

begin training...
            =======  Training  ======= 



RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/new_python_for_gnn/毕设code/language_train/../language_model/FastText.py", line 51, in forward
    out = self.fc1(out)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/linear.py", line 96, in forward
    return F.linear(input, self.weight, self.bias)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py", line 1847, in linear
    return torch._C._nn.linear(input, weight, bias)
Runt

In [31]:
print('begin tesing...')
time7 = time.time()
test_model(fasttext, os.path.join(model_path, model_name + '.pt'),
           test_iter, test_price_iter, model_name)
time8 = time.time()
print('testing time:', time8 - time7)

begin tesing...

            =======  Testing  ======= 



RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/new_python_for_gnn/毕设code/language_train/../language_model/FastText.py", line 51, in forward
    out = self.fc1(out)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/linear.py", line 96, in forward
    return F.linear(input, self.weight, self.bias)
  File "/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py", line 1847, in linear
    return torch._C._nn.linear(input, weight, bias)
Runt

In [1]:
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from torchtext.legacy.data import Iterator
from torch.utils.data import DataLoader
import dill
import os
import time
import sys

sys.path.append('.')
from Data import MyTextDataset, get_field, get_pre_vector, PriceDataset
from utils_dl import train_model, test_model, get_parameter_number, company_deal
from language_model.FastText import Model
from utils import path

import warnings

warnings.filterwarnings("ignore")

path = path.rsplit('/', 1)[0]

In [2]:
print('receiving data path...')
train_data = os.path.join(path, 'data', 'train_set')
validation_data = os.path.join(path, 'data', 'validation_set')
test_data = os.path.join(path, 'data', 'test_set')
#读取数据
print('reading data...')
start_time = time.time()
with open(train_data, 'rb') as f:
    train_example = dill.load(f)
with open(validation_data, 'rb') as f:
    validation_example = dill.load(f)
with open(test_data, 'rb') as f:
    test_example = dill.load(f)
end_time = time.time()
print('data reading time:', end_time - start_time)

receiving data path...
reading data...
data reading time: 1046.096774816513


In [6]:
train_example[12].label

0

In [7]:
NEWS, LAST_NEWS_PERIOD, LABEL = get_field()
fields = [('news', NEWS), ('news_period', LAST_NEWS_PERIOD),
              ("label", LABEL)]

In [10]:
train_set = MyTextDataset(examples=train_example + validation_example,
                              fields=fields)
test_set = MyTextDataset(examples=test_example, fields=fields)

In [13]:
print('feature selection...')
time9 = time.time()
train_set, test_set = company_deal(train_set, test_set)
time10 = time.time()
print('feature selection time:', time10 - time9)

feature selection...
train_set...


In [ ]:
print('receiving pretrained vectors...')
vectors = get_pre_vector() 

In [ ]:
time3 = time.time()
NEWS.build_vocab(
    train_set, test_set, vectors=vectors
)  #去除频率小于十次的词,这会导致一部分资讯没有内容，因此后来不保留这个min_fre(最一开始有的），也能留下比较好的特征
#NEWS.vocab.vectors.unk_init = init.xavier_uniform_
weight_vector = NEWS.vocab.vectors
time4 = time.time()
print('building vocab time:', time4 - time3)
print('矩阵的维度为', weight_vector.shape)

In [ ]:
print('define iterators...')
BATCH_SIZE = 4096
train_iter = Iterator(train_set,
                      batch_size=BATCH_SIZE,
                      device=-1,
                      sort_key=lambda x: len(x.news),
                      sort_within_batch=False,
                      shuffle=False,
                      repeat=False)
test_iter = Iterator(test_set,
                     batch_size=BATCH_SIZE,
                     device=-1,
                     sort=False,
                     sort_within_batch=False,
                     shuffle=False,
                     repeat=False)

In [ ]:
print('price dataset...')
train_price = PriceDataset(train_or_test='train')
test_price = PriceDataset(train_or_test='test')
#构建迭代器
print('price iterators...')
train_price_iter = DataLoader(train_price, batch_size=BATCH_SIZE, shuffle=False)
test_price_iter = DataLoader(test_price, batch_size=BATCH_SIZE, shuffle=False)

In [3]:
print('define model parameter...')
n_vocab = len(NEWS.vocab)
embed_dims = 300
hidden_size = 256
num_classes = 2
weight_matrix = weight_vector
model_name = 'FastText'
n_epochs = 30  #先小点做测试
lr_ = 0.03
weight_decay_ = 0.0003
model_path = '/new_python_for_gnn/毕设code/model_cache'

define model parameter...


NameError: name 'NEWS' is not defined

In [ ]:
print('define model...')
fasttext = Model(n_vocab=n_vocab,
                 embed_dims=embed_dims,
                 hidden_size=hidden_size,
                 num_classes=num_classes,
                 weight_matrix=weight_matrix)
print('模型结构:')
print(fasttext)
parameters_num = get_parameter_number(fasttext)
print('模型总参数量为:')
print('total:{},size:{}M'.format(parameters_num['Total'],
                                 parameters_num['Total'] / 1024**2))
print('模型可训练参数量为:')
print('train:{},size:{}M'.format(parameters_num['Trainable'],
                                 parameters_num['Trainable'] / 1024**2))
#定义训练部件
print('define optimizer and criterion...')
optimizer = optim.Adam(fasttext.parameters(),
                       lr=lr_,
                       weight_decay=weight_decay_)
criterion = F.cross_entropy

In [ ]:
print('begin training...')
time5 = time.time()
train_model(fasttext, train_iter, train_price_iter, optimizer, criterion,
            n_epochs, model_name)
time6 = time.time()
print('training time:', time6 - time5)

In [ ]:
print('begin tesing...')
time7 = time.time()
test_model(fasttext, os.path.join(model_path, model_name + '.pt'),
           test_iter, test_price_iter, model_name)
time8 = time.time()
print('testing time:', time8 - time7)